# Collecte des données des stations concernées.

**Auteur :**  Steve Caron  
**Date de création :** 2023/08/10  
**Présentation :** Ce notebook permet de telecharger des données climatique depuis une liste de stations stockée dans un table BigQuerry et d'envoyer les données en format CSV dans un bucket GCP.


**Prérequis :** 
- Creer un service account pour donner les autorisations nécessaires à l'utilisation de ce script  
  Stocker la clé sour la forme d'un fichier JSON.
- Un bucket gcp pour le stockage de données. (BUCKET_NAME)
- Une table BigQuerry contenant la liste des station (TABLE_ID)
- Un token API infoclimat

**Inputs :** 

**Params:**
* BUCKET_NAME : nom du bucket GCP, DOIT être dans le même projet.
* TABLE_IDE : ID de la table BigQuerry, DOIT être dans le même projet.
* DEPARTEMENT: numéro du département étudié
* NOM_RESEAU: nom du réseau de station étudié
* ANNEE_DEBUT_RECHERCHE: année à partir de laquelle rechercher des donnés.
* ANNEE_FIN_RECHERCHE: année à laquelle arreter de rechercher des données.
* TOKEN : Token de l'api infoclimat. Vous pouvez obtenir un TOKEN depuis https://www.infoclimat.fr/opendata/
* KEY_PATH : Chemin vers le fichier JSON permettant de ce connecter au service account

In [14]:
from google.cloud import bigquery, storage
from google.oauth2 import service_account
import requests
import pandas
import db_dtypes
import csv
import logging

In [15]:
BUCKET_NAME = "code_de_source_lake"
TABLE_ID = "code-de-source.donnees_code_de_source.stations_meteo"
DEPARTEMENT = 66
NOM_RESEAU = "infoclimat.fr"
ANNEE_DEBUT_RECHERCHE = 2001
ANNEE_FIN_RECHERCHE = 2023
TOKEN = "Emp8A4J9Pk587RT3M5SwlhQ4jk3VtHVt0Qlq4XritihpvshdM7BVg"
KEY_PATH = "C:/Service_Account/code-de-source-fb924386ca17"


BUCKET_URI = f"gs://{BUCKET_NAME}"
OUTPUT_PATH= f"{BUCKET_URI}/output/"

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Parametrage du logging

In [16]:
logging.basicConfig(filename='journal_extraction_donnees.log',
                    encoding='utf-8',
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%d/%m/%Y %H:%M:%S ',
                    level=logging.INFO)

# Récupération de nos données depuis BigQuery

In [17]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

sql = """
    SELECT *
    FROM `{}`
""".format(TABLE_ID)

toutes_stations = client.query(sql).to_dataframe()
toutes_stations.head(5)


,nom_station,code_station,departement,pays,coordonnees_x,coordonnees_y,source
0,Embrun,07591,05,FR,6.503,44.567000,METEO-FRANCE
1,Nice - Côte d'Azur,07690,06,FR,7.200,43.653000,METEO-FRANCE
2,Saint-Girons - Antichan,07627,09,FR,1.107,43.001999,METEO-FRANCE
3,Troyes-Barberey,07168,10,FR,4.023,48.327999,METEO-FRANCE
4,Millau - Soulobres,07558,12,FR,3.020,44.118000,METEO-FRANCE


# Récupération des stations du 66 et du réseau infoclimat

Avec Pandas: 

In [18]:
stations = toutes_stations[(toutes_stations["source"] == NOM_RESEAU) &
                            (toutes_stations["pays"] == "FR") &
                            (toutes_stations["departement"] == str(DEPARTEMENT))]

stations.head(15)

,nom_station,code_station,departement,pays,coordonnees_x,coordonnees_y,source
611,Perpignan - Moulin à Vent,000Q3,66,FR,2.895410,42.697640,infoclimat.fr
612,Cabestany,000BS,66,FR,2.941220,42.681320,infoclimat.fr
613,Osséja,000EN,66,FR,1.981920,42.413830,infoclimat.fr
614,[MAE] Lycée Pablo Picasso - PERPIGNAN,ME040,66,FR,2.926642,42.696989,infoclimat.fr
615,[MAE] Lycée Agricole de Théza - Théza,ME121,66,FR,2.938229,42.645670,infoclimat.fr
616,Ste-Colombe-de-la-Commanderie,000QW,66,FR,2.724936,42.617677,infoclimat.fr
617,Font-Romeu-Odeillo-Via,000RX,66,FR,2.059862,42.515899,infoclimat.fr
618,Calce,000TB,66,FR,2.753870,42.758160,infoclimat.fr


In [19]:
liste_stations=stations["nom_station"].tolist()
liste_codes = stations["code_station"].tolist()
print(liste_codes)

['000Q3', '000BS', '000EN', 'ME040', 'ME121', '000QW', '000RX', '000TB']


# Requete des données

In [20]:
def requete(url):

    reponse=requests.get(url)

    return reponse

# Séparation du contenu de la réponse par ligne

In [21]:
def splitlines (reponse):

    #Je dédode mon contenue en UTF8
    contenu = reponse.content.decode('UTF-8')
    #Je split les ligne de mon contenue
    contenu_lignes = contenu.splitlines(True)
    nombre_lignes_reponse = (len(contenu_lignes))

    return contenu_lignes, nombre_lignes_reponse


# Nettoyage des données

La requete renvoie un texte, dons les 6 premières ligne ne contiennent pas de données la 5 ème lignes contient elle le header du csv. 

On cherche donc a récupérer la ligne du header (indice 4) ansi que les lignes contenant les données (indice 6 et plus)

In [22]:
def nettoyage (contenu_sous_forme_ligne):
    
    # Je supprime les lignes qui ne m'interesse pas
    del contenu_sous_forme_ligne[0:4]
    del contenu_sous_forme_ligne[1]

    return contenu_sous_forme_ligne


# Ecriture du fichier en CSV.

In [23]:
def ecriture_en_csv (contenu_sous_forme_ligne_nettoye, nom_fichier):
    
    # Je lis mon fichier avec un csv redear en précisant le délimiteur
    reader = csv.reader(contenu_sous_forme_ligne_nettoye, delimiter=';')
    # J'ouvre le fichier dans lequel je vais sauvegardes mes données
    with open("./data_brut/"+nom_fichier,'w', newline='') as fichier:
        # J'instancie mon objet writer pour écrire un fichier CSV
        writer = csv.writer(fichier,delimiter=',')
        # J'écris mon fichier ligne par ligne
        for line in reader:
            writer.writerow(line)


# Ecriture dans un Bucket

In [24]:
def upload_fichier_dans_bucket(nom_bucket, nom_fichier_source,credentials):
    """Cette fonction prend un fichier enregistrer dans le dossier data_brut et le télécharge dans un bucket GCP"""

    # J'instancie un client Storage
    storage_client = storage.Client(credentials=credentials, project=credentials.project_id,)
    # Je renseigne le nom de mon Bucket
    bucket = storage_client.bucket(nom_bucket)
    # Je creer un blob avec le chemin qu'aura le fichier dans mon bucket
    blob = bucket.blob(f"output/{nom_fichier_source}")
    # J'enregistre le chemin de mon fichier source dans une variable
    chemin_fichier = f"./data_brut/{nom_fichier_source}"

    # J'ouvre mon fichier pour le lire en binaire
    with open (chemin_fichier, "rb") as fichier:
        #J'upload le contenu de mon fichier dans mon blob
        blob.upload_from_file(fichier)

    print(
        f"Le fichier {nom_fichier_source} a été upload dans le bucket {nom_bucket}."
    )

# Ecriture de la main fonction

In [25]:
def main(code,annee):
    
    url=f"https://www.infoclimat.fr/opendata/?method=get&format=csv&stations[]={code}&start={annee}-01-01&end={annee}-12-31&token="+TOKEN
    nom_fichier = f"météo_{code}_{annee}.csv"

    
    reponse = requete(url)

    if reponse.status_code == 200:
        
        contenu_lignes, nombre_lignes_de_la_reponse = splitlines(reponse)

        if nombre_lignes_de_la_reponse > 6:

            contenu_nettoye = nettoyage(contenu_lignes)
            ecriture_en_csv(contenu_nettoye, nom_fichier)
            upload_fichier_dans_bucket(BUCKET_NAME,nom_fichier,credentials)
            logging.info("Données de la station %s pour l'année %s enregistrées avec succès", code, annee)
        
        else:

            print(f"La station {code} n'a pas enregistré de données pour l'année {annee}")
            logging.info("Données de la station %s pour l'année %s pas enregistées car pas de données", code, annee)

    else:

        print("Station {} Année {} Code:{}, la connexion api n'est pas établie".format(code, annee, reponse.status_code))
        logging.error("Données de la station %s pour l'année %s pas enregistées car code erreur %s", code, annee, reponse.status_code )    

# Execution de la fonction

In [26]:
for code in liste_codes:

    for annee in range(ANNEE_DEBUT_RECHERCHE,ANNEE_FIN_RECHERCHE+1):
        main(code,annee)

La station 000Q3 n'a pas enregistré de données pour l'année 2001
La station 000Q3 n'a pas enregistré de données pour l'année 2002
La station 000Q3 n'a pas enregistré de données pour l'année 2003
La station 000Q3 n'a pas enregistré de données pour l'année 2004
La station 000Q3 n'a pas enregistré de données pour l'année 2005
La station 000Q3 n'a pas enregistré de données pour l'année 2006
La station 000Q3 n'a pas enregistré de données pour l'année 2007
La station 000Q3 n'a pas enregistré de données pour l'année 2008
La station 000Q3 n'a pas enregistré de données pour l'année 2009
La station 000Q3 n'a pas enregistré de données pour l'année 2010
La station 000Q3 n'a pas enregistré de données pour l'année 2011
La station 000Q3 n'a pas enregistré de données pour l'année 2012
Le fichier météo_000Q3_2013.csv a été upload dans le bucket datalake_infoclimat.
Le fichier météo_000Q3_2014.csv a été upload dans le bucket datalake_infoclimat.
Le fichier météo_000Q3_2015.csv a été upload dans le bucke